In [6]:
import pandas as pd
from itertools import product
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from rapidfuzz.distance import Levenshtein
import plotly.graph_objects as go
from typing import Dict, Tuple, List, Set

RAN_FIRST_CELL = True

StageInfoEntry = Tuple[str, int, int]
StageInfo = List[StageInfoEntry]
Passwords = Set[str]

def add_stage_info(stage_info: StageInfo, stage: str, user_passwords_dict: Dict[str, Passwords]):
    num_users = len(user_passwords_dict)
    num_passwords = sum(len(passwords) for passwords in user_passwords_dict.values())
    for i, (stage_name, _, _) in enumerate(stage_info):
        if stage_name == stage: # already processed this stage, don't add its 
            stage_info[i] = (stage, num_users, num_passwords)
            return
    stage_info.append((stage, num_users, num_passwords))

def pretty_print_stage_info(stage_info: StageInfo):
    df = pd.DataFrame(stage_info, columns=["Stage", "Num users", "Num passwords"])
    # print with commas to delimit thousands
    print(df.to_string(formatters={"Num users": "{:,}".format, "Num passwords": "{:,}".format}))

def get_pairs_with_edit_distance(user_passwords: Tuple[str, Passwords]) -> List[Tuple[str, str, str, int]]:
    user, passwords = user_passwords
    pairs_with_edit_distance = []
    for pwd1, pwd2 in tqdm(product(passwords, repeat=2), desc=f"Calculating edit distance for {user}"):
        edit_distance = Levenshtein.distance(pwd1, pwd2)
        if edit_distance <= 4 and edit_distance > 0:
            pairs_with_edit_distance.append((user, pwd1, pwd2, edit_distance))
    return pairs_with_edit_distance

def create_sankey(stage_info: StageInfo, data_type="passwords"):
    # Define the data
    data = stage_info
    data = data + [("Final Data", data[-1][1], data[-1][2])] # Adjust sankey to show data filtering in the final stage

    # Prepare the labels (stage names)
    labels = [d[0] for d in data]

    # Prepare the source and target for users and data flows
    source = []
    target = []
    user_values = []
    data_values = []

    for i in range(len(data) - 1):
        # Source to target for both users and data
        source.append(i)  # current stage
        target.append(i + 1)  # next stage
        if data_type == "passwords":
            data_values.append(data[i][2])
        elif data_type == "users":
            # user_values.append(data[i][1])
            user_values.append(data[i][1])
    
    # Create the Sankey diagram
    fig = go.Figure(go.Sankey(
        node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = labels
        ),
        link = dict(
        source = source + source,  # users and data are linked between stages
        target = target + target,
        value = user_values + data_values,  # flows are combined here
        color = ['rgba(44, 160, 44, 0.8)' for _ in user_values] + ['rgba(31, 119, 180, 0.8)' for _ in data_values],  # colors for each type
        )))

    fig.update_layout(title_text="Sankey Diagram - Data and User Flow per Stage", font_size=10)
    fig.show()
    
    
from collections import deque
def write_passwords_with_bfs(guesses_dict, filename="output.txt"):
    # sort users by number of remaining guesses (ascending)
    # this will ensure users with fewer guesses are guessed first
    # to maximize the number of users guessed before reaching budget
    guesses_dict = dict(
    sorted(guesses_dict.items(), key=lambda kv: len(kv[1]))
    )

    active = deque([u for u, q in guesses_dict.items() if q])
    count = 0
    with open(filename, "a", encoding="utf-8", errors="replace") as f:
        while active:
            user = active.popleft()
            q = guesses_dict[user]
            f.write(f"{user}:{q.popleft()}\n")
            count += 1
            if q:               # still has items? requeue for next round
                active.append(user)
    print(f"Passwords written to {filename}, number of rows written: {count}")
    


In [27]:
assert RAN_FIRST_CELL, "Please run the first cell before running this cell"
from tqdm.auto import tqdm

path = "./51k_training_set.txt"
stage_info = [] # format (stage_name, num_users, num_passwords)

# Read in the data, handling impoperly formatted data

## print first 10 lines of the file
user_password_pairs =[]
num_no_colon_lines = 0
with open(path, "r") as f:
    for i, line in tqdm(enumerate(f), desc="Reading in data", total=177000):
        try:
            # strip newline from end of line
            line = line.strip('\n') # only remove newline since passwords can have leading/trailing spaces
            # split line into user and password
            if ":" not in line:
                num_no_colon_lines += 1
                continue
            user, password = line.split(":", 1) # only split on first colon to allow for colons in password
            user_password_pairs.append((user, password))
        except ValueError as e:
            print(f"Error on line {i}: {line}")
            print(f"Error: {e}")
            break  
    
# Update stage info by computing user and password counts
num_unique_users = len(set(user for user, _ in user_password_pairs))
num_password_pairs = len(user_password_pairs)
stage_info.append(("Initial Data", num_unique_users, num_password_pairs))
# print(stage_info)

# Get sets of passwords for each user
from collections import defaultdict
from tqdm.auto import tqdm

user_passwords_dict = defaultdict(set)
for user, password in tqdm(user_password_pairs, desc="Creating user password dictionary"):
    user_passwords_dict[user].add(password)

add_stage_info(stage_info, "Unique Users", user_passwords_dict)

# Filter out users with less than 2 passwords
filtered_user_passwords = {user: passwords for user, passwords in user_passwords_dict.items() if len(passwords) >= 2}
add_stage_info(stage_info, "Users with 2+ Passwords", filtered_user_passwords)

# write function that filters passwords based on custom criteria. Remember to include a return statement and tqdm -_-
def filter_passwords(user_passwords_dict, filter_func):
    output = {}
    for user, passwords in tqdm(user_passwords_dict.items(), desc=f"Filtering passwords: {filter_func.__name__}", total=len(user_passwords_dict)):
        # only remove illegal passwords, keep users with their other legal passwords
        filtered_passwords = {pwd for pwd in passwords if filter_func(pwd)}
        if filtered_passwords:
            output[user] = filtered_passwords
    return output

## Only ASCII
def is_ascii(password):
    return all(ord(char) < 128 for char in password)
filtered_user_passwords = filter_passwords(filtered_user_passwords, is_ascii)
add_stage_info(stage_info, "ASCII Passwords", filtered_user_passwords)

## Excessively long passwords
def is_not_too_long(password):
    return len(password) <= 32
filtered_user_passwords = filter_passwords(filtered_user_passwords, is_not_too_long)
add_stage_info(stage_info, "Passwords <= 32 characters", filtered_user_passwords)

## Excessively short passwords
def is_not_too_short(password):
    return len(password) >= 4
filtered_user_passwords = filter_passwords(filtered_user_passwords, is_not_too_short)
add_stage_info(stage_info, "Passwords >= 4 characters", filtered_user_passwords)

pretty_print_stage_info(stage_info)


Filtering passwords: is_not_too_short: 100%|██████████| 51052/51052 [00:00<00:00, 562963.33it/s]

                        Stage Num users Num passwords
0                Initial Data    51,054       176,297
1                Unique Users    51,054       176,297
2     Users with 2+ Passwords    51,054       176,297
3             ASCII Passwords    51,054       176,295
4  Passwords <= 32 characters    51,052       176,282
5   Passwords >= 4 characters    50,997       174,964


## Find Insights From Data We Removed
```
    # find users that have been removed in this stage, to find insights from data we removed
    removed_users = set(user_passwords_dict.keys()) - set(filtered_user_passwords.keys())
    # write to removed_users.txt
    with open("removed_users.txt", "w") as f:
        for user in removed_users:
            f.write(f"{user}: {user_passwords_dict[user]}\n")
```

Based on observation, we expanded legal password length from 20 to 32. From this, more username-similar passwords were taken into account. In addition, a commonly used hash-value-similar password and its close transformation were found: {"827ccb0eea8a706c4c34a16891f84e", "827ccb0eea8a706c4c34a16891f84e7", "827ccb0eea8a706c4c34a16891f84e7b"}


In [17]:
import hashcat_rule_gen

# load Raj's Best64 rules
rajBest64 = hashcat_rule_gen.load_rules("./Raj'sBest64.rule")

# count number of usage of each rule
rule_usage_count = {str(rule): 0 for rule in rajBest64}
for user, passwords in tqdm(filtered_user_passwords.items(), desc="Counting rule usage"):
    for pwd in passwords:
        for rule in rajBest64:
            new_pwd = hashcat_rule_gen.apply_rule(pwd, rule)
            if new_pwd != pwd and new_pwd in passwords:
                rule_usage_count[str(rule)] += 1
# sort rules by usage count
sorted_rules = sorted(rule_usage_count.items(), key=lambda x: x[1], reverse=True)

# write sorted_rules to a file
with open("best64_sorted_unparsed.rule", "w") as f:
    # write only the rules, not the counts
    for rule, count in sorted_rules:
        f.write(f"{rule}\n")


Counting rule usage: 100%|██████████| 50997/50997 [00:07<00:00, 6732.75it/s] 


## Attention 
This output file "best64_sorted_unparsed.rule" is not in compatible format with hashcat_rule_gen.load_rules(). We used ChatGPT to manually parse it to compatible format. The parsed file is called "best64_sorted.rule" 

In [ ]:

from collections import defaultdict, deque
from tqdm.auto import tqdm
import hashcat_rule_gen

# delete final_output.txt if it exists
import os
if os.path.exists("final_output.txt"):
    os.remove("final_output.txt")
# Load sorted best64 rules
rules64 = hashcat_rule_gen.load_rules("./best64_sorted.rule")


common_passwords = ["12345", "123456789", "1q2w3e", "qwerty", "qwerty123"]
weird_set_1 = {"827ccb0eea8a706c4c34a16891f84e", "827ccb0eea8a706c4c34a16891f84e7", "827ccb0eea8a706c4c34a16891f84e7b"}
# For uniqueness + order: keep both a set and a list
new_passwords_list = defaultdict(list)
seen_passwords = defaultdict(set)

# convert dict items to list for iteration
all_user_items = list(filtered_user_passwords.items())



# Create deque for BFS
deque_passwords_dict = {user: deque(lst) for user, lst in new_passwords_list.items()}
# write initial weird guesses to file
write_passwords_with_bfs(deque_passwords_dict, filename="final_output.txt")
# flush new_passwords_list, but keep seen_passwords
# ensure only those users who use weird passwords are guessed and written to output first while avoiding re-guessing the same passwords
new_passwords_list = defaultdict(list)

# process all users
for user, passwords in tqdm(all_user_items, desc="Generating new passwords for users"):
    # Mail.ru-specific common-password injection
    if user.endswith("@mail.ru"):
        # if length of any password from their password list is greater than 12 
        if any(len(pwd) > 12 for pwd in passwords):
            # try guessing their username as password
            base_username = user.split("@", 1)[0].strip()
            if base_username:
            # try guessing their username as password or username with first letter capitalized
                if base_username not in passwords and base_username not in seen_passwords[user]:
                    seen_passwords[user].add(base_username)
                    new_passwords_list[user].append(base_username)
                capitalized_user = base_username[0].upper() + base_username[1:] if len(base_username) > 0 else base_username
                if capitalized_user not in passwords and capitalized_user not in seen_passwords[user]:
                    seen_passwords[user].add(capitalized_user)
                    new_passwords_list[user].append(capitalized_user)
                no_dash_user = base_username.replace("-", "")
                if no_dash_user not in passwords and no_dash_user not in seen_passwords[user]:
                    seen_passwords[user].add(no_dash_user)
                    new_passwords_list[user].append(no_dash_user)
        # inject common passwords if not already present
        for common_password in common_passwords:
            if common_password not in passwords and common_password not in seen_passwords[user]:
                seen_passwords[user].add(common_password)
                new_passwords_list[user].append(common_password)

    # Apply rules for all users
    if any(len(pwd) > 12 for pwd in passwords):
        # try guessing their username as password or username with first letter capitalized or username removing any '-' and stripping the "@domain" part if present

        base_username = user.split("@", 1)[0].strip()
        if base_username:
        # try guessing their username as password or username with first letter capitalized
            if base_username not in passwords and base_username not in seen_passwords[user]:
                seen_passwords[user].add(base_username)
                new_passwords_list[user].append(base_username)
            capitalized_user = base_username[0].upper() + base_username[1:] if len(base_username) > 0 else base_username
            if capitalized_user not in passwords and capitalized_user not in seen_passwords[user]:
                seen_passwords[user].add(capitalized_user)
                new_passwords_list[user].append(capitalized_user)
            no_dash_user = base_username.replace("-", "")
            if no_dash_user not in passwords and no_dash_user not in seen_passwords[user]:
                seen_passwords[user].add(no_dash_user)
                new_passwords_list[user].append(no_dash_user)

    # for common_password in common_passwords:
    #        if common_password not in passwords and common_password not in seen_passwords[user]:
    #             seen_passwords[user].add(common_password)
    #             new_passwords_list[user].append(common_password)

    # prioritize passwords ending in 1, 2, or 3 and shorter in length
    passwords = sorted(passwords, key=lambda x: (x[-1].isdigit() and x[-1] in "123", len(x)), reverse=True)

    for password in passwords:
        # if password in common_passwords then skip rule application for mail.ru users
        if password in common_passwords and user.endswith("@mail.ru"):
            continue
        for rule in rules64:
            try:
                new_password = hashcat_rule_gen.apply_rule(password, rule)
                if new_password and new_password not in passwords:
                    if len(new_password) > 2 and new_password not in seen_passwords[user]:
                        seen_passwords[user].add(new_password)
                        new_passwords_list[user].append(new_password)
            except Exception:
                continue

# guess the weird passwords first: if any user has any password from weird_set_1, try guessing the other passwords from weird_set_1 first
for user, passwords in tqdm(all_user_items, desc="trying weird1 passwords"):
    # for each user that have at least one password from weird1, add remaining weird passwords to seen_passwords and new_passwords_list
    for pwd in passwords:
        if user.endswith("@mail.ru"):
            if pwd in weird_set_1:
                # for other weird passwords in weird1, if not in passwords, add to seen_passwords and new_passwords_list
                for other_weird_pwd in weird_set_1:
                    if other_weird_pwd == pwd:
                        continue
                    if other_weird_pwd not in passwords and other_weird_pwd not in seen_passwords[user]:
                        seen_passwords[user].add(other_weird_pwd)
                        new_passwords_list[user].append(other_weird_pwd)

# Convert per-user lists to deques (preserve insertion order)
deque_passwords_dict = {user: deque(lst) for user, lst in new_passwords_list.items()}
# append the rest guesses to final_output.txt
write_passwords_with_bfs(deque_passwords_dict, filename="final_output.txt")


Passwords written to final_output.txt, number of rows written: 0


trying weird1 passwords: 100%|██████████| 50997/50997 [00:00<00:00, 673053.01it/s]


Passwords written to final_output.txt, number of rows written: 5868338
